<a href="https://colab.research.google.com/github/atharva-mohite/Assignment-ML_Level-1/blob/main/Atharva_Mohite_Assignment_ML_Level_1_Ans_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question-1: Automating Mobile Camera Calibration

In [1]:
import os
import re
import numpy as np
import cv2 as cv
from google.colab import files
from google.colab.patches import cv2_imshow

In [2]:
uploaded = files.upload()
images = list(uploaded.keys())

Saving cali1.png to cali1.png
Saving cali2.jpg to cali2.jpg
Saving cali3.png to cali3.png
Saving cali4.jpeg to cali4.jpeg
Saving cali5.jpg to cali5.jpg


In [10]:
########## FIND CHESSBOARD CORNERS objPoints AND imgPoints ###########
chessboardSize = (8, 6)
objPoints = []  # 3d point in real world space
imgPoints = []  # 2d points in image plane

# Increase the number of iterations and decrease epsilon for better accuracy
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.0001)

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ...., (7,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboardSize[0], 0:chessboardSize[1]].T.reshape(-1, 2)

for i in range(len(images)):
  img = cv.imread(images[i])
  if i==0:
    height, width = img.shape[:2]
    frameSize = (width, height)
  gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
  gray = cv.equalizeHist(gray)

  ret, corners = cv.findChessboardCorners(gray, chessboardSize, cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_FAST_CHECK + cv.CALIB_CB_NORMALIZE_IMAGE) # Find the chess board corners

  # If found, add object points, image points (after refining them)
  if ret == True:
    objPoints.append(objp)
    corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
    imgPoints.append(corners2)
    cv.drawChessboardCorners(img, chessboardSize, corners2, ret)
    # cv2_imshow(img)

print("Shape of objPoints:", np.array(objPoints).shape)
print("Shape of imgPoints:", np.array(imgPoints).shape)

Shape of objPoints: (5, 48, 3)
Shape of imgPoints: (5, 48, 1, 2)


In [4]:
ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objPoints, imgPoints, frameSize, None, None)

print("Camera calibration parameters of the mobile phone camera are: ")
print("\nRMS re-projection error: ", ret)
print("\nCamera Matrix:\n", cameraMatrix)
print("\nDistortion Parameters: \n", dist)
print("\nRotation Vectors: \n", rvecs)
print("\nTranslation Vectors: \n", tvecs)

Camera calibration parameters of the mobile phone camera are: 

RMS re-projection error:  0.77360268402692

Camera Matrix:
 [[3.47071569e+03 0.00000000e+00 1.95593625e+03]
 [0.00000000e+00 3.48789014e+03 1.18069330e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Distortion Parameters: 
 [[ 2.46216220e-01 -1.72655773e+00  1.07916812e-02 -2.50794120e-03
   4.24211511e+00]]

Rotation Vectors: 
 (array([[ 0.23592183],
       [ 0.03252566],
       [-0.03851247]]), array([[-0.14861482],
       [ 0.03318904],
       [ 1.15727975]]), array([[ 0.17781409],
       [-0.07172579],
       [ 1.50403027]]), array([[ 0.17386235],
       [ 0.02125026],
       [-0.0571129 ]]), array([[ 0.21410919],
       [-0.02814125],
       [ 0.77394754]]))

Translation Vectors: 
 (array([[ 6.22690183],
       [ 2.13801967],
       [32.73285181]]), array([[-2.51588339],
       [-3.91535892],
       [30.55835069]]), array([[-4.59640497],
       [-5.88949892],
       [30.37642415]]), array([[ 4.84526648],
      

In [11]:
for i in range(len(images)):
    img = cv.imread(images[i])
    h, w = img.shape[:2]
    newCameraMatrix, roi = cv.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))

    # Undistort
    dst = cv.undistort(img, cameraMatrix, dist, None, newCameraMatrix)


    combined_img_height = max(img.shape[0], dst.shape[0]) # Dynamically determine combined image height based on input images
    combined_img = np.zeros((combined_img_height, img.shape[1] + dst.shape[1], 3), dtype=np.uint8) # Create a blank image with space for both images
    combined_img[:img.shape[0], :img.shape[1]] = img # Place the original image on the left
    combined_img[:dst.shape[0], img.shape[1]:img.shape[1] + dst.shape[1]] = dst # Place the undistorted image on the right
    # cv2_imshow(combined_img) # Display the combined image

    # Save the undistorted image
    saved_img_name = re.sub(r'\.(png|jpg|jpeg)$', r'_result.\1', images[i])
    cv.imwrite(saved_img_name, dst)

In [12]:
# Reprojection Error
mean_error = 0
for i in range(len(objPoints)):
    imgPoints2, _ = cv.projectPoints(objPoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
    error = cv.norm(imgPoints[i], imgPoints2, cv.NORM_L2) / len(imgPoints2)
    mean_error += error
print("Total error: {}".format(mean_error / len(objPoints)))

Total error: 0.11090760185580392


In [13]:
for image_path in images:
  if os.path.exists(image_path):
      os.remove(image_path)
  if os.path.exists(re.sub(r'\.(png|jpg|jpeg)$', r'_result.\1', image_path)):
      os.remove(re.sub(r'\.(png|jpg|jpeg)$', r'_result.\1', image_path))